In [3]:
from space.planets import venus
from orbit.pointing import simulation
from orbit.geometry import getTiming
from orbit.envision import loadSV
from orbit.elasticOrbit import getSVFromTime, client
from TCNerror import guidanceCovariance, getOrbitCovariances
from measurement.measurement import state_vector
from itertools import product

""" System Packages """
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import json
from datetime import datetime as dt
import os

In [4]:
sim_folder = os.path.join(r"C:\Users",
                          r"ishuwa.sikaneta",
                          r"OneDrive - ESA",
                          r"Documents",
                          r"ESTEC",
                          r"Envision",
                          r"PointingSimulations")

# Simulate Pointing Errors

The following class simulates pointing errors for an Envision-type SAR

In [5]:
mode = {
    "StandardSAR": {
        "axes": np.eye(3),
        "off_nadirs": [18.7, 27.8, 36.9, -18.7, -27.8, -36.9]
    },
    "Altimeter": {
        "axes": np.array([[0,1,0],[1,0,0],[0,0,1]]),
        "off_nadirs": [0.001]
    },
    "Near-Nadir Radiometry": {
        "axes": np.eye(3),
        "off_nadirs": [-14.28]
    },
    "Nadir Radiometry": {
        "axes": np.array([[0,1,0],[1,0,0],[0,0,1]]),
        "off_nadirs": [0.001]
    }
}

### Define Covariances

Define the set of input covariance matrices for the simulation. These are calculated using the guidanceCovariance function

In [6]:
def guidance(tag, off_nadir):
    gError = {"Scenario": getOrbitCovariances(tag),
            "off_nadir": off_nadir,
            "mode": modestr, 
            "contributors": []}
    sv = getSVFromTime(gError["Scenario"]["UTC_time"], frame="VME2000")
    X = sv.measurementData[0]
    rng, _, inc, _, _ = getTiming(sv, [np.radians(off_nadir)])
    gError["range"] = rng[0]
    gError["incidence"] = inc[0]


    R_TCNt, R_TCNp, R_TCNv = guidanceCovariance(tag)
    R_AEUt = eSim.timing2aeuCovariance(X, off_nadir, R_TCNt)
    R_AEUp = eSim.xtrackOffset2aeuCovarianceTCN(X, off_nadir, R_TCNp)
    R_AEUv = eSim.velocity2aeuCovariance(X, off_nadir, R_TCNv)


    """ RPY 2-sigma values from T error ESOC """
    R_RPYt = eSim.aeu2rpyCovariance(R_AEUt)
    contributor = {"Source": "Along-track Error",
                "Domain": "RPY",
                "n_sigma": 2,
                "values": (np.sqrt(np.diag(R_RPYt))*1e3*2).tolist()}
    gError["contributors"].append(contributor)

    """ RPY 2-sigma values from velocity errors ESOC """
    R_RPYv = eSim.aeu2rpyCovariance(R_AEUv)
    contributor = {"Source": "TCN Velocity Error",
                "Domain": "RPY",
                "n_sigma": 2,
                "values": (np.sqrt(np.diag(R_RPYv))*1e3*2).tolist()}
    gError["contributors"].append(contributor)

    """ RPY 2-sigma values from across-track and radial errors ESOC """
    R_RPYp = eSim.aeu2rpyCovariance(np.diag([0.0, R_AEUp, 0.0]))
    contributor = {"Source": "TCN Across-Track Error",
                "Domain": "RPY",
                "n_sigma": 2,
                "values": (np.sqrt(np.diag(R_RPYp))*1e3*2).tolist()}
    gError["contributors"].append(contributor)

    """ Total in the RPY domain """
    rpy_total = (np.sqrt(np.diag(R_RPYt+R_RPYv+R_RPYp))*1e3*2).tolist()
    aeu_total = (np.sqrt(np.diag(R_AEUt+R_AEUv+R_AEUp))*1e3*2).tolist()
     
    for axis, val in zip(["roll","pitch","yaw","azimuth","elevation","tilt"],
                         rpy_total + aeu_total):
        gError["total_%s" % axis] = {"n_sigma": 2, "value": val}
    
    return gError

In [10]:
modestr = "StandardSAR"
eSim = simulation(planet = venus(),
                  e_ang = 14.28, # The effective angle of the beam relative to spacecraft coordinate frame
                  azAxis = 5.5,  # The length of the reflector in the azimuth direction. Leads to 0.99 degrees
                  elAxis = 0.6125,  # The length of the reflector in the elevation direction. Leads to 8.9 degrees
                  carrier = 3.15e9,
                  mode = mode[modestr]["axes"])

In [25]:
tags = ["F1", "F6", "E1", "E6", "L2", "L6", "AWC", 
        "F1r", "F6r", "E1r", "E6r", "L2r", "L6r", "AWCr"]

for tag, off_nadir in tqdm(product(tags, mode[modestr]["off_nadirs"])):
    gError = guidance(tag, off_nadir)
    docid = "%s_%s_%s%0.1f" % (modestr, tag, "p" if off_nadir>0 else "n", np.abs(off_nadir))
    resp = client.index(index="grounderror", id=docid, document=gError)

14it [04:06, 17.59s/it]


In [11]:
off_nadir

0.0

In [29]:
modestr = "Altimeter"
eSim = simulation(planet = venus(),
                  e_ang = 14.28, # The effective angle of the beam relative to spacecraft coordinate frame
                  azAxis = 5.5,  # The length of the reflector in the azimuth direction. Leads to 0.99 degrees
                  elAxis = 0.6125,  # The length of the reflector in the elevation direction. Leads to 8.9 degrees
                  carrier = 3.15e9,
                  mode = mode[modestr]["axes"])
gError = guidance("AWC", 0.000)
gError

C:\Users\ishuwa.sikaneta\OneDrive - ESA\Documents\ESTEC\sarsim\radar\orbit\orientation.py:509: RuntimeWarning: invalid value encountered in sqrt
  s2 = look*w*np.sqrt((1-v**2)*wwQ - (w*v*sinG)**2)/wwQ
C:\Users\ishuwa.sikaneta\OneDrive - ESA\Documents\ESTEC\sarsim\radar\orbit\orientation.py:510: RuntimeWarning: invalid value encountered in sqrt
  s2F = look*wF*np.sqrt((1-v**2)*wwQ - (w*v*sinG)**2)/wwQ


[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]


LinAlgError: Array must not contain infs or NaNs

In [27]:
R_TCNt, R_TCNp, R_TCNv = guidanceCovariance("AWC")

In [28]:
R_TCNp

array([[5.48840635e-11, 0.00000000e+00],
       [0.00000000e+00, 4.91449668e+00]])